# Adult income classification

In the following notebook we'll be using the "Adult" dataset. This dataset contains a binary label indicating if a person's annual income is larger than $50k per year. The data is available on the [UCI machine learning repository](https://archive.ics.uci.edu/ml/index.php).

In [1]:
import pandas as pd

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
names = [
    'age', 'workclass', 'fnlwgt', 'education',
    'education-num', 'marital-status', 'occupation',
    'relationship', 'race', 'gender', 'capital-gain',
    'capital-loss', 'hours-per-week', 'native-country',
    'salary'
]
dtypes = {
    'workclass': 'category',
    'education': 'category',
    'marital-status': 'category',
    'occupation': 'category',
    'relationship': 'category',
    'race': 'category',
    'gender': 'category',
    'native-country': 'category'
}

X = pd.read_csv(url, names=names, header=None, dtype=dtypes)
y = X.pop('salary').map({' <=50K': False, ' >50K': True})

X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


`ethik` analyzes a model based on the predictions it makes on a test set. Consequently we first have to split our dataset in two.

In [2]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, shuffle=True, random_state=42)

We will now train a model using [LightGBM](https://lightgbm.readthedocs.io/en/latest/).

In [3]:
import lightgbm as lgb

model = lgb.LGBMClassifier(random_state=42).fit(X_train, y_train)

We can now make predictions for the test set. We'll use a variable named `y_pred` to store the predicted probabilities associated with the `True` label.

In [4]:
# We use a named pandas series to make plot labels more explicit
y_pred = pd.Series(model.predict_proba(X_test)[:, 1], name='>$50k')

We can now fit an `Explainer` using the features from the test set. This will analyze the distribution of each feature and build a set of `lambda` coefficients which can be used to explain model predictions.

In [5]:
import ethik

explainer = ethik.Explainer()

## Understanding model predictions

We can start by looking at how the probability of having a higher income changes with respect to the `education-num` variable, **as perceived by the model**.

In [6]:
explainer.plot_bias(
    X_test=X_test['education-num'],
    y_pred=y_pred,
    inline=True # We are in a notebook
)

Clearly we see that the model believes that the probability of having a salary above $50k increases with the amount of education. Although this might seem like an obvious statement, it's good to confirm that the model is seeing it. Moreover, it's helpful to be able to quantify by how much the model changes it's predictions.

We can also plot the distribution of predictions for more than one variable. However, because different variables have different scales we have to use a common measure to display them together. For this purpose we plot the τ ("tau") values. These values are contained between -1 and 1 and simply reflect by how much the variable is shifted from it's mean towards it's lower and upper quantiles. In the following figure a tau value of -1 corresponds to just under 20 years old whereas a tau value of 1 refers to being slightly over 60 years old.

In [7]:
explainer.plot_bias(
    X_test=X_test[['age', 'hours-per-week', 'education-num']],
    y_pred=y_pred,
    inline=True
)

One of the uses of these kinds of plots is to see if variables affect the outcome on average or not. Indeed, the straighter the lines, the less the associated variable has an impact on the average outcome. This is very handy to know if said variable is, say, a social trait such as the ethinicity and the target is a credit score. In this case, `ethik` can be used to visualize and quantify the bias of the model with respect to the social trait.

We can also get an overview of features' importance and determine which ones impact the predictions the most:

In [8]:
explainer.plot_bias_ranking(
    X_test=X_test[['age', 'hours-per-week', 'education-num']],
    y_pred=y_pred,
    inline=True
)

The importance is computed as the average absolute difference in bias changes per tau increase. If the curves plotted above are horizontal lines, we can conclude that the corresponding features do not impact the predictions at all. So, to compute the importance of a feature, we compute the distance to this horizontal line.

## Evaluating model reliability

`ethik` can also be used to assess the reliability of a model with respect to a variable. Let us first evaluate the global performance of our model.

In [9]:
from sklearn import metrics

# As `y_test` is binary (0 or 1), we need to make `y_pred` binary as well
# for `metrics.accuracy_score` to work.
print(f'Accuracy score: {metrics.accuracy_score(y_test, y_pred > 0.5):.4f}')

Accuracy score: 0.8788


With `ethik` we can see how performant the model is with respect to a variable, for example `age`.

In [10]:
explainer.plot_performance(
    X_test=X_test['age'],
    y_test=y_test,
    y_pred=y_pred > 0.5,
    metric=metrics.accuracy_score,
    inline=True
)

We can see that although the overall accuracy is around 0.88, it is much higher when the age is lower. This is quite intuitive, as we can imagine that young adults more often than not have a salary under $50k. When they get older, many things can happen and their salary isn't as easy to guess, which translates to a lower model accuracy.

In the same way as before, we can visualize the performance of the metric with respect to multiple variables. 

In [11]:
explainer.plot_performance(
    X_test=X_test[['age', 'education-num']],
    y_test=y_test,
    y_pred=y_pred > 0.5,
    metric=metrics.accuracy_score,
    inline=True
)

These kind of reliability plots can be used for many cases. For example, now that we know the model is less accurate for older people than young ones, we might want to focus our data analysis on older people in order to extract helpful features. `ethik` can thus help guide a data science project by telling you where your model is failing to perform.  

We can also rank features by their impact on performance. Here, we want to show how bad the model can be when we make each feature's mean change:

In [12]:
explainer.plot_performance_ranking(
    X_test=X_test[['age', 'education-num']],
    y_test=y_test,
    y_pred=y_pred > 0.5,
    metric=metrics.accuracy_score,
    criterion="min", # We are looking at the worst accuracy
    inline=True
)

This plot tells us that whatever the mean age of the dataset is (among the ones we computed), the model's accuracy is at least 86%. We also notice that changing the mean of `education-num` can lead to worse performance than changing the mean age.

## Categorical features

Until now, we just manipulated numeric features. But ethik can also compute the influence of categorical features, that must have either the type `object` or `category` in the dataframe:

In [13]:
X_test['gender']

14160     Female
27048     Female
28868       Male
5667      Female
7827        Male
          ...   
26658       Male
29401       Male
27265     Female
30339       Male
8610        Male
Name: gender, Length: 8141, dtype: category
Categories (2, object): [Female, Male]

In [14]:
explainer.plot_bias(
    X_test=X_test['gender'],
    y_pred=y_pred,
    inline=True
)

From the categorical feature `gender`, two numeric features are created: one per category, which represents the proportion of this category in the dataset (between 0 and 1). Since we only have two possible values for `gender`, the resulting numeric features are symetric.

Not surprisingly, but sadly, we can see that the more the men in the dataset, the higher the proportion of individuals earning more than $50k a year. Let's still notice that this is a correlation and that causality is not proved at this stage.

We can plot the performance as well:

In [15]:
explainer.plot_performance(
    X_test=X_test['gender'],
    y_test=y_test,
    y_pred=y_pred > 0.5,
    metric=metrics.accuracy_score,
    inline=True
)

## Robustness

Right. We can observe both the model bias and performance but are these observations reliable? One criterion to trust an algorithm is its robustness, *i.e.* the fact that it gives similar outputs for similar inputs.

To check the robustness in `ethik`, we can compute a confidence interval on the explanation:

1. Get `p`% of the lines in the dataset
2. Compute the explanation (*i.e.* the bias or the performance)
3. Do that `n` times
4. Have a look at the distribution of the explanations

Let's do it for `n = 30` and `p = 0.8` (the default):

In [16]:
# Please check the API reference for thinner configuration
explainer = ethik.Explainer(n_samples=30)

Then we can explain and plot the bias simply as before. To compute the confidence interval, we consider the 5% and 95% quantiles (this can be configured, see API reference):

In [17]:
# For readability reasons, the confidence interval is displayed
# when there is one variable only
explainer.plot_bias(
    X_test=X_test['education-num'],
    y_pred=y_pred,
    inline=True
)

The plotted line is the mean. The confidence interval is so small that we need to zoom in to see it. This means that the algorithm is quite robust on this dataset (which only contains the `education-num` feature).